In [12]:
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

True

In [2]:
# Load your CSV files
patients = pd.read_csv("../data/patients.csv")
print(f"Shape of 'patients': {patients.shape}")
physicians = pd.read_csv("../data/physicians.csv")
print(f"Shape of 'physicians': {physicians.shape}")
visits = pd.read_csv("../data/visits.csv")
print(f"Shape of 'visits': {visits.shape}")
hospitals = pd.read_csv("../data/hospitals.csv")
print(f"Shape of 'hospitals': {hospitals.shape}")
reviews = pd.read_csv("../data/reviews_cleaned.csv")
print(f"Shape of 'reviews_cleaned': {reviews.shape}")
payers = pd.read_csv("../data/payers.csv")
print(f"Shape of 'payers': {payers.shape}")

Shape of 'patients': (9649, 5)
Shape of 'physicians': (500, 6)
Shape of 'visits': (9998, 15)
Shape of 'hospitals': (30, 3)
Shape of 'reviews_cleaned': (1005, 3)
Shape of 'payers': (5, 2)


In [4]:
print("patients: " + str(patients.columns.tolist()))
print("physicians: " + str(physicians.columns.tolist()))
print("visits: " + str(visits.columns.tolist()))
print("hospitals: " + str(hospitals.columns.tolist()))
print("reviews: " + str(reviews.columns.tolist()))
print("payers: " + str(payers.columns.tolist()))

patients: ['patient_name', 'patient_sex', 'patient_id', 'patient_dob', 'patient_blood_type']
physicians: ['physician_name', 'physician_id', 'physician_dob', 'physician_grad_year', 'medical_school', 'salary']
visits: ['patient_id', 'date_of_admission', 'billing_amount', 'room_number', 'admission_type', 'discharge_date', 'test_results', 'visit_id', 'physician_id', 'payer_id', 'hospital_id', 'chief_complaint', 'treatment_description', 'primary_diagnosis', 'visit_status']
hospitals: ['hospital_id', 'hospital_name', 'hospital_state']
reviews: ['review_id', 'visit_id', 'review']
payers: ['payer_name', 'payer_id']


In [14]:
visits.shape # focus on "visits" because it is the fact tables

(9998, 15)

In [17]:
visits.isnull().sum()

patient_id                  0
date_of_admission           0
billing_amount              0
room_number                 0
admission_type              0
discharge_date            500
test_results                0
visit_id                    0
physician_id                0
payer_id                    0
hospital_id                 0
chief_complaint          6998
treatment_description    6998
primary_diagnosis        6998
visit_status                0
dtype: int64

In [20]:
print(reviews.shape)
reviews.isnull().sum()

(1005, 3)


review_id    0
visit_id     0
review       0
dtype: int64

In [22]:
# Find visit_ids in reviews that are not in visits
invalid_visit_ids = reviews[~reviews['visit_id'].isin(visits['visit_id'])]

# Show result
print("Visit IDs in reviews but not in visits:")
print(invalid_visit_ids['visit_id'].unique())

# Optional: show full rows from reviews
print("\nFull rows in reviews with invalid visit_id:")
print(invalid_visit_ids)

Visit IDs in reviews but not in visits:
[]

Full rows in reviews with invalid visit_id:
Empty DataFrame
Columns: [review_id, visit_id, review]
Index: []


In [50]:
# Merge data into one denormalized DataFrame
# data = visits.merge(patients, on='patient_id', how='left') \
#     .merge(physicians, on='physician_id', how='left') \
#     .merge(hospitals, on='hospital_id', how='left') \
#     .merge(payers, on='payer_id', how='left') \
#     .merge(payers, on='payer_id', how='left') \
#     .merge(reviews, on='visit_id', how='left')
# print(data.shape)
# print("patient_name" in data.columns.tolist())
# data.head()

In [51]:
# data.isnull().sum()

In [33]:
def check_duplicated(data):    
    duplicated_counts = {}
    for col in data.columns:
        duplicated = data[col].duplicated(keep=False)  # count all duplicates (not just first)
        count = duplicated.sum()
        if count > 0:
            duplicated_counts[col] = count
        else:
            duplicated_counts[col] = 0

    for col, count in duplicated_counts.items():
        print(f"Column '{col}' has {count} duplicated values")

In [34]:
check_duplicated(visits)

Column 'patient_id' has 1141 duplicated values
Column 'date_of_admission' has 9949 duplicated values
Column 'billing_amount' has 0 duplicated values
Column 'room_number' has 9998 duplicated values
Column 'admission_type' has 9998 duplicated values
Column 'discharge_date' has 9931 duplicated values
Column 'test_results' has 9998 duplicated values
Column 'visit_id' has 0 duplicated values
Column 'physician_id' has 9998 duplicated values
Column 'payer_id' has 9998 duplicated values
Column 'hospital_id' has 9998 duplicated values
Column 'chief_complaint' has 9995 duplicated values
Column 'treatment_description' has 9996 duplicated values
Column 'primary_diagnosis' has 9997 duplicated values
Column 'visit_status' has 9998 duplicated values


In [42]:
check_duplicated(reviews)

Column 'review_id' has 0 duplicated values
Column 'visit_id' has 99 duplicated values
Column 'review' has 357 duplicated values


In [52]:
# check_duplicated(data)

In [43]:
duplicate_review_visits = reviews['visit_id'].value_counts()
duplicates = duplicate_review_visits[duplicate_review_visits > 1]

print(f"Number of duplicated visit_ids in reviews: {len(duplicates)}")
print("Example duplicated visit_ids:\n", duplicates.head())

Number of duplicated visit_ids in reviews: 48
Example duplicated visit_ids:
 visit_id
9363    3
6659    3
9911    3
6662    2
7312    2
Name: count, dtype: int64


In [44]:
reviews[reviews['visit_id'] == 9363]

,review_id,visit_id,review
541,300,9363,I had a mixed experience at the hospital. The ...
542,426,9363,"The hospital staff was accommodating, but the ..."
543,586,9363,I had a mixed experience at the hospital. The ...


In [48]:
# Step 1: Concatenate all reviews per visit_id
review_texts = reviews.groupby('visit_id')['review'].apply(lambda x: ' | '.join(x))

# Step 2: For each visit_id, get the first review_id (or you could use min, max, etc.)
first_review_ids = reviews.groupby('visit_id')['review_id'].first()

# Step 3: Combine both into a single DataFrame
reviews_grouped = pd.DataFrame({
    'review_id': first_review_ids,
    'review': review_texts
}).reset_index()

print(reviews_grouped.shape)
reviews_grouped.head()

(954, 3)


,visit_id,review_id,review
0,4,377,I am grateful for the attentive care I receive...
1,5,861,"The medical care I received was excellent, and..."
2,9,655,"The hospital staff was friendly and helpful, b..."
3,55,85,I had a challenging experience at the hospital...
4,63,929,"The hospital provided excellent care, but the ..."


In [49]:
check_duplicated(reviews_grouped)

Column 'visit_id' has 0 duplicated values
Column 'review_id' has 0 duplicated values
Column 'review' has 292 duplicated values


In [54]:
data = visits.merge(patients, on='patient_id', how='left') \
    .merge(physicians, on='physician_id', how='left') \
    .merge(hospitals, on='hospital_id', how='left') \
    .merge(payers, on='payer_id', how='left') \
    .merge(payers, on='payer_id', how='left') \
    .merge(reviews_grouped, on='visit_id', how='left')
print(data.shape)
# print("patient_name" in data.columns.tolist())

(9998, 30)


In [55]:
def row_to_doc(row):
    return f"""
    Visit ID: {row.get('visit_id', 'Unknown')}
    Patient: {row.get('patient_name', 'Unknown')}, Sex: {row.get('patient_sex', 'Unknown')}, DOB: {row.get('patient_dob', 'Unknown')}, Blood Type: {row.get('patient_blood_type', 'Unknown')}
    Physician: {row.get('physician_name', 'Unknown')} (Graduated: {row.get('physician_grad_year', 'Unknown')} from {row.get('medical_school', 'Unknown')})
    Hospital: {row.get('hospital_name', 'Unknown')}, State: {row.get('hospital_state', 'Unknown')}
    Complaint: {row.get('chief_complaint', 'Unknown')}
    Diagnosis: {row.get('primary_diagnosis', 'Unknown')}
    Treatment: {row.get('treatment_description', 'Unknown')}
    Test Results: {row.get('test_results', 'Unknown')}
    Admission Date: {row.get('date_of_admission', 'Unknown')} -> Discharge: {row.get('discharge_date', 'Unknown')}
    Billing: ${row.get('billing_amount', 'Unknown')}, Room: {row.get('room_number', 'Unknown')}, Payer: {row.get('payer_name', 'Unknown')}
    Review: {row.get('review', 'No review provided')}
    """

documents = data.apply(row_to_doc, axis=1).tolist()
len(documents)

9998

In [56]:
print(documents[100])


    Visit ID: 83
    Patient: Kimberly Vargas, Sex: Female, DOB: 1948-10-26, Blood Type: AB-
    Physician: Nathan Smith (Graduated: 1993-08-12 from Johns Hopkins University School of Medicine)
    Hospital: Boyd PLC, State: GA
    Complaint: Memory decline and perplexity
    Diagnosis: F01.9 - Vascular dementia, unspecified
    Treatment: Cognitive assessments, medication management, and lifestyle changes. Identify and address the underlying cause of cognitive decline.
    Test Results: Abnormal
    Admission Date: 2018-12-28 -> Discharge: 2019-01-27
    Billing: $5319.542762776323, Room: 139, Payer: Unknown
    Review: nan
    


In [60]:
print(documents[4])


    Visit ID: 4
    Patient: Mrs. Brandy Flowers, Sex: Male, DOB: 1989-01-26, Blood Type: AB+
    Physician: Mark Vang (Graduated: 1987-08-11 from University of California, San Francisco School of Medicine)
    Hospital: Vaughn PLC, State: CO
    Complaint: Persistent cough and shortness of breath
    Diagnosis: J45.909 - Unspecified asthma, uncomplicated
    Treatment: Prescribed a combination of inhaled bronchodilators (such as albuterol) and corticosteroids to manage airway inflammation. Conducted pulmonary function tests to assess lung function. Administered antibiotics if a bacterial infection was suspected. Advised lifestyle modifications, including smoking cessation and environmental control.
    Test Results: Normal
    Admission Date: 2021-07-09 -> Discharge: 2021-08-02
    Billing: $18086.34418356388, Room: 477, Payer: Unknown
    Review: I am grateful for the attentive care I received during my stay. The hospital environment was calm and conducive to healing.
    
